## Problem Statement
We are going to solve an image classifiction problem, by predicting whether an image is a T-shit, a shirt, a dress, or something else. 
By doing so, we would be training a deep neural network using TensorFlow and Keras to recognize the types of clothes. <br>

The plan for our project is
- First, we download the dataset and use a pretrained model to classify images.
- Then, we talk about neural networks, and see how they work internally.
- After that, we adjust the pretrained neural network for solving our tasks.
- Finally, we expand our dataset by generating many more images from the images
we have.

For evaluating the quality of our models, let’s use accuracy: the percentage of items we
classified correctly

In [ ]:
## download the clothing dataset
!git clone https://github.com/alexeygrigorev/clothing-dataset-small.git

In [ ]:
## lets import our libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import tensorflow as tf
from tensorflow import kerasas 

### Loading Images

In [ ]:
##import the library
from tensorflow.keras.preprocessing.image import load_img



In [ ]:
##load the images 
load_img(fullname, target_size=(299, 299))

## Convolutional neural networks
A convolutional neural network extracts visual patterns from an image and use them for classification. 

### Using a pretrained model 
Pretrained models are a type of network architecture that has already been trained on a particular type of dataset(ImageNet) and can be used for general-purpose image classification.  

Loading our model 
We specify two parameters here:
 weights: We want to use a pretrained model from ImageNet.
 imput_shape: The size of the input images: height, width, and the number of
channels. We resize the images to 299 × 299, and each image has three channels: red, green and blue.

In [ ]:
## lets import our pretrained model and other helpful functions 
from tensorflow.keras.applications.xception import Xception, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions


In [ ]:
## lets load the images from disk in small batches
train_gen = ImageDataGenerator(
    ##lets apply the preprocess_input function to each image
    preprocessing_function=preprocess_input
)

In [ ]:
## lets point the train_gen to the directory with the data

train_ds = train_gen.flow_from_directory(
    ##loads all the images from the train dir
    "clothing-dataset-small/train/",
    
    ##lets resize the images 
    target_size=(150, 150),
    
    ##lets load the images in batches
    batch_size=32,
    
)

Lets repeart same for the validation set

In [ ]:
valid_gen = ImageDataGenerator(
    
    preprocessing_function=preprocess_input
)

valid_ds = valid_gen.flow_from_directory(

    "clothing-dataset-small/validation/",
    target_size=(150, 150),
    batch_size=32,
    
)

### Creating the model
We begin by loading the base model which is the pretrained model that we're using for extracting the vector representation from the images by including the part with pretrained convolutional layers. 
Next we add our own dense layers. <br>
By seeting the include_top parameter to False, we explicitly specify that we're not interested in the dense layers of the pretrained neural network, only in the convolutional layers. <br>
We avoid training the base mode by setting the trainable parameter to False. This freezes the base model and attempting to train the base_model would destroy all the filter. <br>

Building the base model: 
we would use functional styling to build the model
- creating the base model
- we use the base model as a function by giving it two parameters: inputs, and training=False
- we create a pooling layer: which is a special construction that allows us to convert the output of a convolutional layer into a vector and immediately invoke it with base as the argument. meaning the input to this layer comes from base. 
- we create a dense layer and connect it with vector. 
- we wrap the inputs and outputs into a Model class by specifying two things
what the model will get as input,
what the output of the model is.

In [ ]:
## lets create the base model
base_model = Xception(
    
    ##lets use the pretrained model on ImageNet
    weights='imagenet',
    
    ##keep only the convolutional layers
    include_top=False,
    
    ## images should be 150 x 150 with three channels
    input_shape=(150, 150, 3),

)

## lets freeze the base model 
base_model.trainable = False

In [ ]:
##lets specify the input and the size of the array
inputs = keras.Input(shape=(150, 150, 3))

## create the base model 
## lets extract high-level features 
base = base_model(inputs, training=False)

##lets extract the vector representation by converting the output of the base_model to a vector
vector = keras.layers.GlobalAveragePooling2D()(base)

##adds a dense layer of size 10: one element for each class
outputs = keras.layers.Dense(10)(vector)

## combines the inputs and the outputs into a keras model
model = keras.Model(inputs, outputs)



### Training the model


In [ ]:
##setting the learning rate
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate)

In [ ]:
learning_rate = 0.01
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']

)


In [ ]:
model.fit(train_ds, epochs=10, validation_data=valid_ds)